In [1]:
import tensorflow as tf

In [2]:
# Reward system
REWARD_FOR_BUILDING_SEQUENCE = 5
REWARD_FOR_EXPOSING_HIDDEN_CARD = 10
REWARD_FOR_CREATING_EMPTY_PILE = 15
REWARD_FOR_BUILDING_ON_HIGHER_CARD_OUT_OF_SUIT = 2
REWARD_FOR_MAXIMIZING_CARD_EXPOSURE_BEFORE_NEW_DEAL = 20
REWARD_FOR_NATURAL_BUILD_POST_SUIT_REMOVAL = 10

# Penalties
PENALTY_FOR_REDUNDANT_MOVE = -10

# Game mechanics
MAXIMUM_EMPTY_PILES = 10  # Assuming Spider Solitaire with 10 tableau piles
MAX_CARDS_PER_SUIT = 13  # Number of cards per suit, from Ace to King

# Neural network parameters - These can be adjusted based on the model's performance and complexity
# Placeholder, adjust based on actual game state representation
NN_INPUT_SHAPE = (10, 10)
NN_LEARNING_RATE = 0.001
NN_BATCH_SIZE = 32
NN_EPOCHS = 100

# Other constants
# Adjust this for training with different difficulty levels (1, 2, 3, or 4 suits)
NUMBER_OF_SUITS = 1
SUITS = ["C", "D", "S", "H"]
RANKS = ["A", "2", "3", "4", "5", "6", "7", "8", "9", "T", "J", "Q", "K"]
CARD_COUNT = 104

In [3]:
def parse_game_state(file_path):
    with open(file_path, "r") as file:
        lines = file.readlines()

    draw_count = int(lines[0].split(": ")[1])
    card_matrix = [line.strip().split(" ")
                   for line in lines[2:]]  # Skip the first two lines

    # Convert the card matrix into a more structured format, if necessary
    # For now, it's just a list of lists containing the card values or placeholders

    return draw_count, card_matrix

In [4]:
draw_count, card_matrix = parse_game_state("input.txt")
print(draw_count)
print(card_matrix)

50
[['##', '##', '##', '##', '##', '##', '##', '##', '##', '##'], ['##', '##', '##', '##', '##', '##', '##', '##', '##', '##'], ['##', '##', '##', '##', '##', '##', '##', '##', '##', '##'], ['##', '##', '##', '##', 'HJ', 'C2', 'DJ', 'S2', 'D9', 'S4'], ['D9', 'C3', 'SK', 'H8', '__', '__', '__', '__', '__', '__']]


In [5]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Input(shape=NN_INPUT_SHAPE),  # Explicit Input layer
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(optimizer="adam", loss="binary_crossentropy",
              metrics=["accuracy"])

2024-03-26 12:07:30.241598: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2 Max
2024-03-26 12:07:30.241619: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 64.00 GB
2024-03-26 12:07:30.241623: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 24.00 GB
2024-03-26 12:07:30.241640: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-26 12:07:30.241652: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [7]:
import game_generator as gg

# Example of generating game states
num_games = 1000  # Number of games to generate for the dataset
game_states = []
for _ in range(num_games):
    # Or adjust for different levels of difficulty
    deck = gg.create_deck(NUMBER_OF_SUITS)
    deck = gg.shuffle_deck(deck)
    grid, remaining_deck = gg.deal_deck(deck)
    game_str = gg.output_game_as_str(grid)
    game_states.append(gg.GameState(grid, remaining_deck, game_str))


def encode_card(suit, rank):
    suit_id = SUITS.index(suit)
    rank_id = RANKS.index(rank)
    return suit_id * len(RANKS) + rank_id


def decode_card(unique_id):
    suit_id = unique_id // len(RANKS)
    rank_id = unique_id % len(RANKS)
    return SUITS[suit_id], RANKS[rank_id]


def preprocess_game_state(game_str):
    lines = game_str.split("\n")
    # Skip the first line (draw count) and the last (empty)
    card_lines = lines[1:-1]
    encoded_state = []
    for line in card_lines:
        cards = line.split()
        for card in cards:
            encoded_state.extend(encode_card(card))
    return encoded_state


x_train = [preprocess_game_state(game_str) for game_str in game_states]

y_train = []

model.fit(x_train, y_train, epochs=10)

draw: 50
## ## ## ## ## ## ## ## ## ## 
## ## ## ## ## ## ## ## ## ## 
## ## ## ## ## ## ## ## ## ## 
## ## ## ## ## ## ## ## ## ## 
## ## ## ## CK C7 C6 C9 C6 CQ 
CQ C5 C5 C2 __ __ __ __ __ __ 
C5
C7


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
def make_move(model, game_state):
    # This function should use the model to predict the best move
    # Update the game state accordingly
    # For now, this is a placeholder showing the structure
    pass


game_over = True
# Loop to play the game
while not game_over:
    # Assuming a function game_over() to check if the game has ended
    make_move(model, card_matrix)
    # Wait for user to update input.txt and re-parse it
    draw_count, card_matrix = parse_game_state("input.txt")